In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
import os
from newspaper import Article
# pdfminer.six
# import pdfminer
# import pdfminer.high_level
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

In [5]:
headers = {
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh,zh-TW;q=0.9,zh-CN;q=0.8,en;q=0.7',
        'origin': 'https://allafrica.com',
        'referer': 'https://allafrica.com/',
        'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }




from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10


def crawl_news_article(url):
    time.sleep(2)
    try:
        a = Article(url, config= config, keep_article_html=True)
        a.download()
        a.parse()
        news_article = a.text
        if news_article is None:
            print('None\t%s'%url)
    except:
        print('404\t%s'%url)

    return news_article

def extract_text(url):
    try:
        time.sleep(5)
        coverpage = requests.get(url, headers = headers).content
        soup = BeautifulSoup(coverpage, 'html.parser')
    except:
        print('\nerror connection: ', url)
        return
    
    try:
        body = soup.find_all('div',class_= 'story-body')[0]
        find_text = body.find_all('p', class_="story-body-text")
        text = [i.text for i in find_text]
        text = '\n'.join(text)
    except:
#         text = ''
        print('\nNone: ', url)
        try:
            time.sleep(2)
            a = Article(url, config= config, keep_article_html=True)
            a.download()
            a.parse()
            text = a.text
        except:
            text = ''
    return text


In [3]:
pandarallel.initialize(nb_workers=2, progress_bar=False)
batch_size = 2

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
df1 = pd.read_csv('africa_text.csv', header=[0])
df1

url  \
0      https://allafrica.com/stories/201405140640.html   
1      https://allafrica.com/stories/201404301238.html   
2      https://allafrica.com/stories/201203050049.html   
3      https://allafrica.com/stories/201211060895.html   
4      https://allafrica.com/stories/201309250279.html   
...                                                ...   
34990  https://allafrica.com/stories/202103261012.html   
34991  https://allafrica.com/stories/202103261013.html   
34992  https://allafrica.com/stories/202103261017.html   
34993  https://allafrica.com/stories/202103261050.html   
34994  https://allafrica.com/stories/202103270020.html   

                                                                                                                                                 headline  \
0                                                                                                              50 Tiv Villages in Taraba Turn Ghost Towns   
1                                                                         UNHCR Renews Call for Protection of Displaced People After Attack On CAR Convoy   
2                                                                                           TFG Security Forces On High Alert to Thwart Attacks in Baidoa   
3                                                                                                      Boko Haram Accused of Ties With Sahel 'Terrorists'   
4                                                                                  Alakyo Killings - Nasarawa Govt Was Advised to Abort Operation - Ewuga   
...                                                                                                                                                   ...   
34990                            Kenya: Belt and Road in Kenya - Covid-19 Sparks a Reckoning With Debt and Dissatisfaction (Council on Foreign Relations)   
34991                                                                                                     Ministry to Recruit HIV Counsellors for Workers   
34992                                                                                                                CFA Organizes Forum On The New Sudan   
34993  Africa: Foreign Affairs Committee Sends Bass Resolution On COVID Collaboration With Africa To House Floor (United States House Of Representatives)   
34994                                                                                   Multiple People Trapped in Mozambique Hotel After Militant Attack   

                                                                                                                                                                                                                                                                                                                                                                                                                  text  
0      Jalingo — The clashes between suspected Fulani herders andTiv farmers, which started around the Nasarawa and Benue axis, later spilled over to Taraba State with more devastating impacts.\nOver 50 Tiv settlements within the Gassolchiefdom in Gassol local government area Taraba State were said to have been sacked, with scores of people now displaced and seeking refuge in neighbouring localities ...  
1      Bangui, Central African Republic — The UN refugee agency on Tuesday stressed the need to provide physical protection to forcibly displaced communities a day after a fatal grenade attack on a humanitarian convoy.\nThe convoy, carrying members of the Muslim community to safety in Kabo and Moyen Sido in the north, was hit by a grenade believed to have been thrown by an Anti-Balaka militiamen. The...  
2                           Baidoa — The Transitional Federal Government TFG of Somalia said on Saturday that it will tackle security threats from Al-shabab militants against Baidoa town, the capital of Bay region of southern Somalia.\nLt.Ade

In [5]:
df1[df1.text.isnull()]

url  \
5906   https://allafrica.com/stories/200404260126.html   
5907   https://allafrica.com/stories/200501040307.html   
5908   https://allafrica.com/stories/200505050515.html   
5909   https://allafrica.com/stories/200505091158.html   
5910   https://allafrica.com/stories/200506080476.html   
...                                                ...   
34990  https://allafrica.com/stories/202103261012.html   
34991  https://allafrica.com/stories/202103261013.html   
34992  https://allafrica.com/stories/202103261017.html   
34993  https://allafrica.com/stories/202103261050.html   
34994  https://allafrica.com/stories/202103270020.html   

                                                                                                                                                 headline  \
5906                                                                                                                        Rwanda Back in Congo, Says UN   
5907                                                                                           Polio Cases Soar As Conflict Hampers Vaccination Campaigns   
5908                                                                                Aid Projects Threatened As Donors Tire of 'No War No Peace' Situation   
5909                                                                                                                 Angola Refugees Wary of Repatriation   
5910                                                                                   Corruption Trial Pressures Mbeki, Facing G8 Summit, to Fire Deputy   
...                                                                                                                                                   ...   
34990                            Kenya: Belt and Road in Kenya - Covid-19 Sparks a Reckoning With Debt and Dissatisfaction (Council on Foreign Relations)   
34991                                                                                                     Ministry to Recruit HIV Counsellors for Workers   
34992                                                                                                                CFA Organizes Forum On The New Sudan   
34993  Africa: Foreign Affairs Committee Sends Bass Resolution On COVID Collaboration With Africa To House Floor (United States House Of Representatives)   
34994                                                                                   Multiple People Trapped in Mozambique Hotel After Militant Attack   

      text  
5906   NaN  
5907   NaN  
5908   NaN  
5909   NaN  
5910   NaN  
...    ...  
34990  NaN  
34991  NaN  
34992  NaN  
34993  NaN  
34994  NaN  

[29089 rows x 3 columns]

In [8]:
pandarallel.initialize(nb_workers=2, progress_bar=False)
batch_size = 2

i=6000
while i < df1.shape[0]:
    df1.loc[i:i+batch_size-1, 'text']=\
    df1.loc[i:i+batch_size-1, 'url'].parallel_apply(extract_text).values

    print(i, end=',')
    
    if i% 100 ==0:
        df1.to_csv('africa_text.csv', index=False)
        
    i = i + batch_size
    
df1.to_csv('africa_text.csv', index=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
6000,6002,6004,6006,6008,6010,6012,6014,6016,6018,6020,6022,6024,6026,6028,6030,6032,6034,6036,6038,6040,6042,6044,6046,6048,6050,6052,6054,6056,6058,6060,6062,6064,6066,6068,6070,6072,6074,6076,6078,6080,6082,6084,6086,6088,6090,6092,6094,6096,6098,6100,6102,6104,6106,6108,6110,6112,6114,6116,6118,6120,6122,6124,6126,6128,6130,6132,6134,6136,6138,6140,6142,6144,6146,6148,6150,6152,6154,6156,6158,6160,6162,6164,6166,6168,6170,6172,6174,6176,6178,6180,6182,6184,6186,6188,6190,6192,6194,6196,6198,6200,6202,6204,6206,6208,6210,6212,6214,6216,6218,6220,6222,6224,6226,6228,6230,6232,6234,6236,6238,6240,6242,6244,6246,6248,6250,6252,6254,6256,6258,6260,6262,6264,6266,6268,6270,6272,6274,6276,6278,6280,6282,6284,6286,6288,6290,6292,6294,6296,6298,6300,6302,6304,6306,6308,6310,6312,6314,6316,6318,6320,6322,6324,6326,6328,6330,6332,6334,6336,6338,6340,634

In [9]:
quit()

In [123]:
df_all.to_csv('df_finish.csv', index=False)
df_explode.to_csv('df_explode.csv', index=False)

In [4]:
df_all = pd.read_csv('df_finish.csv', header=[0])
df_all[df_all.duplicated(subset='url')]

df_explode = pd.read_csv('df_explode.csv', header=[0])
df_explode[df_explode.duplicated(subset='url')]

Empty DataFrame
Columns: [url, headline, text]
Index: []

In [5]:
df1 = df_explode[~df_explode.url.isin(df_all.url)].reset_index(drop=True)
df1

url  \
0     https://allafrica.com/stories/201209280214.html   
1     https://allafrica.com/stories/201209280328.html   
2     https://allafrica.com/stories/201209280551.html   
3     https://allafrica.com/stories/201209190134.html   
4     https://allafrica.com/stories/201209190640.html   
...                                               ...   
4140  https://allafrica.com/stories/201411130601.html   
4141  https://allafrica.com/stories/201012070659.html   
4142  https://allafrica.com/stories/201501121729.html   
4143  https://allafrica.com/stories/201211030359.html   
4144  https://allafrica.com/stories/201901290691.html   

                                                                          headline  \
0                       Govt's Robust Approach to Boko Haram Paying-Off - Jonathan   
1                             We Are Winning War Against Boko Haram, Says Jonathan   
2                                               Another Bomb Factory Found in Kano   
3                                      Military Force Takes Over Boko Haram Centre   
4          JTF Raid 'Qaqa's Home' in Kano - - Recovers Bombs, Arms and Ammunition.   
...                                                                            ...   
4140                        Female Suicide Bomber Attacks Niger's Teaching College   
4141                 UN - Somalia's New PM Vows to End Child Recruitment Into Army   
4142                                U.S. Condemns Terrorist Attacks in Borno State   
4143                         Govt Withdraws From Peacekeeping Over M23 Accusations   
4144  Revealed - How Al-Shabaab Are Circulating Millions of Terror Money Unnoticed   

     text  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  
...   ...  
4140  NaN  
4141  NaN  
4142  NaN  
4143  NaN  
4144  NaN  

[4145 rows x 3 columns]

In [3]:
df1=pd.read_csv('df1.csv', header=[0])
df1

url  \
0     https://allafrica.com/stories/201209280214.html   
1     https://allafrica.com/stories/201209280328.html   
2     https://allafrica.com/stories/201209280551.html   
3     https://allafrica.com/stories/201209190134.html   
4     https://allafrica.com/stories/201209190640.html   
...                                               ...   
4140  https://allafrica.com/stories/201411130601.html   
4141  https://allafrica.com/stories/201012070659.html   
4142  https://allafrica.com/stories/201501121729.html   
4143  https://allafrica.com/stories/201211030359.html   
4144  https://allafrica.com/stories/201901290691.html   

                                                                          headline  \
0                       Govt's Robust Approach to Boko Haram Paying-Off - Jonathan   
1                             We Are Winning War Against Boko Haram, Says Jonathan   
2                                               Another Bomb Factory Found in Kano   
3                                      Military Force Takes Over Boko Haram Centre   
4          JTF Raid 'Qaqa's Home' in Kano - - Recovers Bombs, Arms and Ammunition.   
...                                                                            ...   
4140                        Female Suicide Bomber Attacks Niger's Teaching College   
4141                 UN - Somalia's New PM Vows to End Child Recruitment Into Army   
4142                                U.S. Condemns Terrorist Attacks in Borno State   
4143                         Govt Withdraws From Peacekeeping Over M23 Accusations   
4144  Revealed - How Al-Shabaab Are Circulating Millions of Terror Money Unnoticed   

                                                                                                                                                                                                                                                                                                                                                                                                                 text  
0     President Goodluck Jonathan told the United Nations Thursday that Nigeria's "robust" approach to neutralizing a threat posed by Islamist sect, Boko Haram using military force, holding indirect talks with the group and improving education in the North is paying off.\nIn an interview with Reuters on the sidelines of the U.N. General Assembly's annual gathering of world leaders, President Goodluc...  
1     President Goodluck Jonathan has expressed optimism that the acts of terrorism being perpetrated by the Islamic sect, Boko Haram, which has killed over 1,400 people in the North since 2010, will soon be over.\nJonathan told Reuters in an interview on Wednesday in New York on the sidelines of the ongoing General Assembly meeting of the United Nations, that the Federal Government's multifaceted a...  
2     Kano — Another local bomb-making factory was yesterday found at Hotoro Haye in Kano by the police during an operation in the area. Kano Commissioner of Police Alhaji Ibrahim K. Idris confirmed the development, saying two suspects were arrested at the factory.\nOther items recovered from the factory include two AK47 rifles, two pistols, two bags of TNT, ready-made IEDs, remote control bombs, ga...  
3     The Military Joint Task Force (JTF) claimed, yesterday, that it had taken over Boko Haram's command centre in Kano, an action that has further dealt a massive blow on the capability of the sect.\nTwo members of the sect had been captured and killed in the early hours of yesterday after a gun battle with the military and officers of the State Security Services (SSS). One of those captured was s...  
4     Kano — Operatives of JTF Monday night in Kano raided the residence of suspected top media coordinator of radical Islamist group at Rimi Kebe and Rijayar'zaki general area of the municipality and recovered arms, ammunition.\nIn what look like a coordinated military operation by the JTF 

In [6]:
pandarallel.initialize(nb_workers=2, progress_bar=False)
batch_size = 2

i=32
while i < df1.shape[0]:
    df1.loc[i:i+batch_size-1, 'text']=\
    df1.loc[i:i+batch_size-1, 'url'].parallel_apply(extract_text).values

    print(i, end=',')
    
    if i% 100 ==0:
        df1.to_csv('df1.csv', index=False)
        
    i = i + batch_size
    
df1.to_csv('df1.csv', index=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200,202,204,206,208,210,212,214,216,218,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,322,324,326,328,330,332,334,336,338,340,342,344,346,348,350,352,354,356,358,360,362,364,366,368,370,372,374,376,378,380,382,384,386,388,390,392,394,396,398,400,402,404,406,408,410,412,414,416,418,420,422,424,426,428,430,432,434,436,438,440,442,444,446,448,450,452,454,456,458,460,462,464,466,468,470,472,474,476,

In [7]:
df1.to_csv('df1.csv', index=False)

# merge

In [39]:
df_all = pd.read_csv('df_finish.csv', header=[0])
df1 = pd.read_csv('df1.csv', header=[0])
df_all.shape, df1.shape

((39952, 3), (4145, 3))

In [40]:
df1[df1.duplicated(subset='url')].shape, df_all[df_all.duplicated(subset='url')].shape

((0, 3), (0, 3))

In [41]:
df_final = pd.concat([df_all, df1]).sort_values('url').reset_index(drop=True)
df_final.to_csv('africa_final.csv', index=False)
df_final

url  \
0      https://allafrica.com/stories/199906250157.html   
1      https://allafrica.com/stories/200009050372.html   
2      https://allafrica.com/stories/200012140390.html   
3      https://allafrica.com/stories/200012180313.html   
4      https://allafrica.com/stories/200311070082.html   
...                                                ...   
44092  https://allafrica.com/stories/202103261013.html   
44093  https://allafrica.com/stories/202103261017.html   
44094  https://allafrica.com/stories/202103261050.html   
44095  https://allafrica.com/stories/202103270020.html   
44096  https://allafrica.com/stories/202103270021.html   

                                                                                                                                                 headline  \
0                                                                                                 Women Group Calls For Intervention To End Somali Strife   
1                                                                                                                         Mali Arrests Gangsters in Mopti   
2                                                                                                Chagossians Seek $6 Billion Compensation From Washington   
3                                                                                                    Mauritius Prepared To Negotiate Diego Garcia With US   
4                                                                                                     Kinshasa Calls for More Support in Landmine Removal   
...                                                                                                                                                   ...   
44092                                                                                                     Ministry to Recruit HIV Counsellors for Workers   
44093                                                                                                                CFA Organizes Forum On The New Sudan   
44094  Africa: Foreign Affairs Committee Sends Bass Resolution On COVID Collaboration With Africa To House Floor (United States House Of Representatives)   
44095                                                                                   Multiple People Trapped in Mozambique Hotel After Militant Attack   
44096                                                                                    We Need Cash to Adapt, Kenyan Slum Dwellers Tell Cop26 President   

                                                                                                                                                                                                                                                                                                                                                                                                                  text  
0                Nairobi, Kenya (PANA) — A Somali Women's group on Friday in Nairobi called\nfor intervention by the international community to resolve the ongoing conflicts\nin the horn of Africa region.\nThe women, under a caucus known as "Somali Women's Campaign for Political\nParticipation" told a news conference in the Kenyan capital that most of the\nproblems in the region were rooted in clannism.  
1                                                                               BAMAKO, Mali — A group of gangsters who used\nto terrorise residents in the Malian towns of Mopti,\nSevare and Konna have been arrested by police.\nThe gangsters were suspected of carrying out several\nkillings, the most recent being the slaying of a Guinean\nnational living in Mopti, a shopkeeper and another man in\nSevare.  
2      Port Louis — The leader of the Chagos\nrefugee group in Mauritius, Olivier Bancoult, said Thursday that\nUS lawyers will ask for six billion dollars compensation from the\nUS government, on behalf of the 6,000 Chagossians residing in\n

In [42]:
df_final[df_final.duplicated(subset='url')].shape

(0, 3)

In [43]:
df_explode = pd.read_csv('africa_story_links.csv', header=[0])
df_final.shape, df_explode.shape                        

((44097, 3), (44097, 3))

In [44]:
df_final[df_final.text.isnull()]

url  \
35394  https://allafrica.com/stories/201710260027.html   
38024  https://allafrica.com/stories/201811080031.html   

                                                                               headline  \
35394  Abubakar Shekau's Wife May Have Been Killed in Airstrike, Nigerian Military Says   
38024                             How You Dey?, Prince Charles Speaks 'Pidgin' in Lagos   

      text  
35394  NaN  
38024  NaN

In [45]:
df_final.headline.str.len().describe()

count    44097.000000
mean        52.153752
std         16.429137
min          6.000000
25%         41.000000
50%         50.000000
75%         61.000000
max        252.000000
Name: headline, dtype: float64

In [46]:
df_final.text.str.len().describe()

count     44095.000000
mean       1045.730536
std        2298.077811
min           9.000000
25%         364.000000
50%         465.000000
75%         670.000000
max      138899.000000
Name: text, dtype: float64

In [48]:
df_final[df_final.text.str.len()<100]

url  \
229    https://allafrica.com/stories/200901300010.html   
669    https://allafrica.com/stories/200907220650.html   
1746   https://allafrica.com/stories/201008260113.html   
1826   https://allafrica.com/stories/201009260009.html   
2123   https://allafrica.com/stories/201012090908.html   
...                                                ...   
37027  https://allafrica.com/stories/201805220117.html   
37272  https://allafrica.com/stories/201806200847.html   
37487  https://allafrica.com/stories/201807110288.html   
37529  https://allafrica.com/stories/201807160404.html   
38247  https://allafrica.com/stories/201812100074.html   

                                                                              headline  \
229                               U.S. Calls On Leaders, People to Avoid More Violence   
669                                           A Timeline of the Abyei Boundary Dispute   
1746             Secretary-General's Remarks to Security Council Open Debate on Piracy   
1826                           U.S. Plans to Strengthen Ties with Somaliland, Puntland   
2123                                            Nation Responds to Sudan Tank Demarche   
...                                                                                ...   
37027                                           The African Union [Podcast Transcript]   
37272                                          President Isaias' Speech On Martyrs Day   
37487       Convening the Electorate for the Election of the President of the Republic   
37529  Speech Delivered By President Isaias Afwerki At Millennium Hall, in Addis Abeba   
38247                                                          Risky Journey to Europe   

                                                                              text  
229               Statement by Robert A. Wood, Acting Spokesman\nMadagascar Crisis  
669    Nairobi — The following is a timeline for the Abyei boundary dispute.\n1905  
1746                                      New York — Mr. President,\nExcellencies,  
1826                  New York — U.S. DEPARTMENT OF STATE\nOffice of the Spokesman  
2123                     S E C R E T SECTION 01 OF 02 NAIROBI 002497\nNOFORNSIPDIS  
...                                                                            ...  
37027                           Peacebuilders  \nEpisode Four: “The African Union”  
37272                            Distinguished Participants\nLadies and Gentlemen,  
37487            Decree N°2018/391 Of 09 July 2018\nThe President Of The Republic,  
37529        Distinguished People of Ethiopia\nYour Excellency, Dr. Abiy Ahmed Ali  
38247      Search for greener pastures carries deadly consequences\nLouise Donavan  

[139 rows x 3 columns]

In [49]:
df_final = df_final[~df_final.text.isnull()]
df_final = df_final[df_final.text.str.len()>100].reset_index(drop=True)
df_final

url  \
0      https://allafrica.com/stories/199906250157.html   
1      https://allafrica.com/stories/200009050372.html   
2      https://allafrica.com/stories/200012140390.html   
3      https://allafrica.com/stories/200012180313.html   
4      https://allafrica.com/stories/200311070082.html   
...                                                ...   
43950  https://allafrica.com/stories/202103261013.html   
43951  https://allafrica.com/stories/202103261017.html   
43952  https://allafrica.com/stories/202103261050.html   
43953  https://allafrica.com/stories/202103270020.html   
43954  https://allafrica.com/stories/202103270021.html   

                                                                                                                                                 headline  \
0                                                                                                 Women Group Calls For Intervention To End Somali Strife   
1                                                                                                                         Mali Arrests Gangsters in Mopti   
2                                                                                                Chagossians Seek $6 Billion Compensation From Washington   
3                                                                                                    Mauritius Prepared To Negotiate Diego Garcia With US   
4                                                                                                     Kinshasa Calls for More Support in Landmine Removal   
...                                                                                                                                                   ...   
43950                                                                                                     Ministry to Recruit HIV Counsellors for Workers   
43951                                                                                                                CFA Organizes Forum On The New Sudan   
43952  Africa: Foreign Affairs Committee Sends Bass Resolution On COVID Collaboration With Africa To House Floor (United States House Of Representatives)   
43953                                                                                   Multiple People Trapped in Mozambique Hotel After Militant Attack   
43954                                                                                    We Need Cash to Adapt, Kenyan Slum Dwellers Tell Cop26 President   

                                                                                                                                                                                                                                                                                                                                                                                                                  text  
0                Nairobi, Kenya (PANA) — A Somali Women's group on Friday in Nairobi called\nfor intervention by the international community to resolve the ongoing conflicts\nin the horn of Africa region.\nThe women, under a caucus known as "Somali Women's Campaign for Political\nParticipation" told a news conference in the Kenyan capital that most of the\nproblems in the region were rooted in clannism.  
1                                                                               BAMAKO, Mali — A group of gangsters who used\nto terrorise residents in the Malian towns of Mopti,\nSevare and Konna have been arrested by police.\nThe gangsters were suspected of carrying out several\nkillings, the most recent being the slaying of a Guinean\nnational living in Mopti, a shopkeeper and another man in\nSevare.  
2      Port Louis — The leader of the Chagos\nrefugee group in Mauritius, Olivier Bancoult, said Thursday that\nUS lawyers will ask for six billion dollars compensation from the\nUS government, on behalf of the 6,000 Chagossians residing in\n

In [50]:
df_final.to_csv('africa_final.csv', index=False)

# final files:

<p>africa_nagivation_links.csv:  each navigation page
<p>africa_story_links.csv: all stories link
<p>africa_final.csv: final text